# Inverted Indices and Boolean Search

In this chapter, we will learn how to manually construct an inverted index in pure Python. To help us in this task we shall make use of the [NLTK](http://nltk.org) natural language processing library.

In [1]:
import nltk
import os

## Choosing a collection of Documents

Collections of documents, also know as *corpora* or *corpus* (in sigular form), are the starting point of any information retrieval task.

Em seguida vamos importar mais coisas necessárias para o nosso trabalho. Note que estamos baixando a obra completa de Machado de Assis, com a qual iremos alimentar nosso índice.

In [2]:
from nltk.corpus import machado, mac_morpho
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
import string
from collections import defaultdict
from nltk.stem.snowball import PortugueseStemmer

Vamos também baixar o banco de *stopwords* do NLTK. Stop words são um conjunto de palavras que normalmente carregam baixo conteúdo semântico e portanto não são alvo de buscas.

In [3]:
nltk.download('stopwords')
nltk.download('machado')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fccoelho/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package machado to /home/fccoelho/nltk_data...
[nltk_data]   Package machado is already up-to-date!


True

In [7]:
textos = [machado.raw(id) for id in machado.fileids()]

In [9]:
textos[0][:80]

'Conto, Contos Fluminenses, 1870\n\nContos Fluminenses\n\nTexto-fonte:\n\nObra Completa'

Lendo o texto *puro* dos livros de Machado:

In [10]:
#textos = [machado.raw(id) for id in machado.fileids()]
#len(textos)

Carregando a  lista de stopwords em lingua portuguesa para limpeza dos textos. Note que é preciso trazer as palavras para *UTF-8* antes de usá-las.

In [11]:
swu = stopwords.words('portuguese') + list (string.punctuation)
#swu = [word.decode('utf8') for word in sw]

In [13]:
stopwords.words('portuguese')[:10]
#list (string.punctuation)

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para']

Um outro ingrediente essencial é um stemmer para a nossa língua. O Stemmer reduz as palavras a uma abreviação que se aproxima da "raiz" da palavra.

In [18]:
stemmer = PortugueseStemmer()

In [19]:
WordPunctTokenizer().tokenize(textos[0])[:10]

['Conto',
 ',',
 'Contos',
 'Fluminenses',
 ',',
 '1870',
 'Contos',
 'Fluminenses',
 'Texto',
 '-']

Preparando o Texto
------------------

Na célula abaixo, vamos normalizar os nossos textos trazendo todas as palavras para caixa baixa e abreviando-as de forma a deixar apenas as suas raízes. Neste passo, removeremos também as *stopwords*. Tenha paciência, esta análise vai levar algum tempo...

In [20]:
textos_limpos = []
for texto in textos:
    tlimpo = [stemmer.stem(token.lower()) for token in WordPunctTokenizer().tokenize(texto) if token not in swu]
    textos_limpos.append(tlimpo)

Vejamos uma amostra do resultado:

In [21]:
textos_limpos[0][150:160]

['uma', 'tal', 'miss', 'doll', 'dev', 'ter', 'poet', 'tennyson', 'cor', 'ler']

Construindo um Índice Invertido
-------------------------------

De posse da nossa lista de termos *normalizados*, podemos agora construir o nosso índice invertido.

In [22]:
indice = defaultdict(lambda:set([]))
for tid,t in enumerate(textos_limpos):
    for term in t:
        indice[term].add(tid)

Podemos verificar a estrutura interna do nosso índice, procurando por uma palavra:

In [23]:
indice[stemmer.stem("Dollar")]

{0, 3}

In [24]:
print(stemmer.stem('Salarial'))

salarial


Vamos ver o contexto em que a palavra *Salário* ocorre em um dos textos

In [25]:
nltk.Text(WordPunctTokenizer().tokenize(textos[182])).concordance("Salário")

Displaying 2 of 2 matches:
operários que com esse acréscimo de salário proporcionariam às suas famílias ma
s 2 horas da sesta é equivalente ao salário de meio dia , em tais casos abonado


In [26]:
def busca(consulta):
    """
    A construção de uma função de busca é deixada com exercício ao leitor
    """
    pass

Mas já podemos utilizar nosso índice diretamente com alguns termos e verificar como o mesmo é eficiente. 

In [28]:
%time
results = indice[stemmer.stem('nacional')]&indice[stemmer.stem('perdi')] - indice[stemmer.stem('campo')]
list(results)[:10]

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 µs


[164, 69, 122, 137, 138, 171, 155, 141, 73, 219]

Para um exame mais preciso do tempo de execução da nossa consulta, podemos usar a mágica do *%%timeit*

In [29]:
%%timeit
results = indice[stemmer.stem('nacional')]&indice[stemmer.stem('perdi')] - indice[stemmer.stem('campo')]
#results

10000 loops, best of 3: 75.7 µs per loop
